##**Notebook PC#11**

## Encoder-Decoder LSTM for Natural Language Processing.

**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Beatriz Akiria de Assis Quaresma - 203899 <br>
**Aluno(a):** Decio Miranda Filho - 236087

In [ ]:
from random import seed
from random import randint
from numpy import array
from numpy import argmax

In [ ]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X,y = list(), list()
    for i in range(n_examples):
        in_pattern=[randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X,y

In [ ]:
seed(1)
n_samples =1
n_numbers = 2
largest = 10
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

[[3, 10]] [13]


In [ ]:
from math import ceil
from math import log10

In [ ]:
def to_string(X,y,n_numbers,largest):
    max_length = n_numbers*ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    maxlength = ceil(log10(n_numbers*(largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' 'for _ in range(maxlength-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [ ]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10

In [ ]:
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

X,y = to_string(X,y,n_numbers,largest)
print(X,y)

[[3, 10]] [13]
[' 3+10'] ['13']


In [ ]:
alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']

In [ ]:
def integer_encode(X,y,alphabet):
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [ ]:
X,y = integer_encode(X,y,alphabet)

In [ ]:
print(X,y)

[[11, 3, 10, 1, 0]] [[1, 3]]


In [ ]:
def one_hot_encode(X,y,max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)

    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [ ]:
X,y = one_hot_encode(X,y,len(alphabet))

In [ ]:
print(X,y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [ ]:
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y

In [ ]:
def invert(seq,alphabet):
    int_to_char = dict((i,c) for i,c in enumerate(alphabet))
    strings  = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [ ]:
n_terms = 3
largest = 10
alphabet = [str(x) for x in range(10)] + ['+', ' ']

In [ ]:
n_chars = len(alphabet)
n_in_seq_length = n_terms*ceil(log10(largest+1)) +n_terms-1
n_out_seq_length = ceil(log10(n_terms*(largest+1)))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length,n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(n_chars,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                26400     
                                                                 
 repeat_vector (RepeatVecto  (None, 2, 75)             0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 2, 50)             25200     
                                                                 
 time_distributed (TimeDist  (None, 2, 12)             612       
 ributed)                                                        
                                                                 
Total params: 52212 (203.95 KB)
Trainable params: 52212 (203.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
X,y = generate_data(75000,n_terms,largest,alphabet)
model.fit(X,y,epochs=1,batch_size=10)

7500/7500 [==============================] - 58s 7ms/step - loss: 0.4256 - accuracy: 0.8721


In [ ]:
X,y = generate_data(100,n_terms,largest,alphabet)
loss,acc = model.evaluate(X,y,verbose=0)
print('Loss: %f, Accuracy: %f' %(loss,acc*100))

Loss: 0.030535, Accuracy: 99.500000


In [ ]:
for _ in range(10):
    X,y = generate_data(1,n_terms,largest,alphabet)
    yhat = model.predict(X,verbose=0)
    in_seq = invert(X[0],alphabet)
    out_seq = invert(y[0],alphabet)
    predicted = invert(yhat[0],alphabet)
    print('%s = %s (expect %s)' %(in_seq,predicted,out_seq))

   3+7+5 = 15 (expect 15)
  2+10+6 = 18 (expect 18)
   4+5+9 = 18 (expect 18)
   2+2+4 =  8 (expect  8)
 10+7+10 = 27 (expect 27)
  10+2+3 = 15 (expect 15)
   8+4+1 = 13 (expect 13)
   8+1+3 = 12 (expect 12)
   5+9+6 = 20 (expect 20)
   1+5+8 = 14 (expect 14)


<font color="green">
Atividade (a) <br>
Como são gerados os dados de treinamento?
</font>

**Resposta:**

Os dados de treinamento são gerados através dos seguintes passos:

1. **Geração de Pares Aleatórios:** A função *random_sum_pairs* gera *n_examples* pares de números inteiros aleatórios, com um valor máximo definido por *largest*. Para cada par a função calcula a soma dos números e armazena o resultado. Assim, para cada entrada (um vetor de números), há uma saída correspondente (a soma desses números).

2. **Conversão para Strings:** A entrada e saída são convertidos em strings formatadas, com números separados por '+' e preenchidos com espaços em branco para padronizar o comprimento.

3. **Codificação Inteira:** As strings são convertidas para sequências de inteiros com base em um alfabeto (de 0 a 9, '+' e ' ')  que mapeia caracteres para inteiros, o *char_to_int*.

4. **Codificação One-Hot:** Por fim, as sequências de inteiros são convertidas em vetores one-hot, que são então usados como dados de entrada e saída para o treinamento do modelo LSTM.

Por exemplo, se chamarmos a função **random_sum_pairs(1, 2, 10)**, ela retornará os seguintes valores:



```
random_sum_pairs(1, 2, 10)
([[7, 9]], [16])
```


Isso significa que temos um único exemplo de entrada dado por um par de números inteiros `[7, 9]` e o resultado esperado é `[16]`.

<font color="green">
Atividade (b) <br>
Como uma calculadora simples pode operar baseada no conceito de tradução de frases, ou seja, sem realizar operações algébricas?
</font>

**Resposta:**

O conceito de tradução de frases na criação de uma calculadora simples pode ser entendido como a conversão das perguntas em forma de frases em sequências de caracteres que representem os números e operadores matemáticos. Em seguida, essas sequências são processadas para fazer a correspondência entre as perguntas e as respostas corretas, como no princípio da Sala Chinesa em Seattle.

A ideia do processo pode ser entendida como:

1. **Entrada do Usuário:** O usuário insere uma expressão matemática, como "3 + 5".
2. **Tokenização:** A expressão é dividida em tokens individuais, como mostrado acima. Por exemplo, `["3", "+", "5"]`.
3. **Encoder LSTM:** Cada token é processado sequencialmente por um bloco LSTM, mapeando a expressão para um estado interno que captura sua semântica.
4. **Decoder LSTM:** O estado interno é passado para um segundo bloco LSTM que gera a frase de saída token por token. Para "3 + 5", a saída seria `["8"]`.

Por exemplo:
- **Entrada:** "7 - 2"
- **Tokens:** `["7", "-", "2"]`
- **Encoder LSTM:** Processa os tokens e gera um estado interno representando "7 menos 2".
- **Decoder LSTM:** Usa o estado interno para gerar a saída `["5"]`.

Assim, a LSTM captura dependências e semânticas em sequências, permitindo que a calculadora "entenda" e "traduza" expressões diretamente em respostas corretas.